In [ ]:
# Testing Cell
from aviary.mission.flight_phase_builder import FlightPhaseOptions
from aviary.mission.height_energy.phases.groundroll_phase import GroundrollPhaseOptions as FGopt
from aviary.mission.solved_two_dof.phases.solved_twodof_phase import SolvedTwoDOFPhaseOptions
from aviary.mission.two_dof.phases.accel_phase import AccelPhaseOptions
from aviary.mission.two_dof.phases.ascent_phase import AscentPhaseOptions
from aviary.mission.two_dof.phases.climb_phase import ClimbPhaseOptions
from aviary.mission.two_dof.phases.cruise_phase import CruisePhaseOptions
from aviary.mission.two_dof.phases.descent_phase import DescentPhaseOptions
from aviary.mission.two_dof.phases.groundroll_phase import GroundrollPhaseOptions as GGopt
from aviary.mission.two_dof.phases.rotation_phase import RotationPhaseOptions
from aviary.utils.doctape import glue_keys

dummy_phase_info = {}
dummy_phase_info.update(AccelPhaseOptions().items())
dummy_phase_info.update(AscentPhaseOptions().items())
dummy_phase_info.update(ClimbPhaseOptions().items())
dummy_phase_info.update(CruisePhaseOptions().items())
dummy_phase_info.update(DescentPhaseOptions().items())
dummy_phase_info.update(FGopt().items())
dummy_phase_info.update(FlightPhaseOptions().items())
dummy_phase_info.update(GGopt().items())
dummy_phase_info.update(RotationPhaseOptions().items())
dummy_phase_info.update(SolvedTwoDOFPhaseOptions().items())
glue_keys(dummy_phase_info)

# Mission Definition
"Mission definition" is all of the information needed for Aviary to create a Dymos trajectory that flies the vehicle through the desired mission profile. Specifically, it defines all of the information needed to define Dymos phases (a list of that information can be found in the [Dymos docs](https://openmdao.org/dymos/docs/1.9.1/api/phase_api.html)). This is a vast list, and is much more detailed than the majority of Aviary users need to know. Aviary only needs a select amount of information about each mission from the user, which is conveyed in the mission definition. This takes the form of a Python dictionary commonly referred to as the `phase_info` in Aviary.

## Phase Info Overview
Aviary uses a nested Python dictionary to define and control all aspects of the mission and its phases. The dictionary must have a very specific structure, and all valid keys Aviary recognizes are detailed later in this section. Understanding this format is necessary if you want to modify existing missions or design your own.

The top level of the `phase_info` contains a name string and a sub-dictionary for each phase in the mission. Users are free to choose any valid python name for their phases (valid characters are upper/lower case letters, numbers, and underscores.) In addition, there are two default categories "pre_mission" and "post_mission". These contain any settings that pertain to the pre_mission, which is any calculation that occurs before the mission runs, and the post_mission, which is any calculation that occurs after the mission runs. The sub-dictionaries for "pre_mission" and "post_mission" must be named exactly as shown, and cannot have custom names.

```python
phase_info = {
    'pre_mission': {},
    'climb1': {},
    'climb2': {},
    'cruise': {},
    'descent': {},
    'post_mission': {},
}
```

Note that the order is important, so "pre_mission" should be first, followed by phases in chronological order, terminated by "post_mission". All of the sections shown in this example are optional, including "pre_mission", and "post_mission" - the options that can be configured in those sections are not required.

```note
When using the Python API, the variable name for the dictionary is not important, since you will just be passing it as an argument to the function that runs Aviary. If you are providing your `phase_info` as a file, such as when running Aviary from the command line, the dictionary _must_ be named "phase_info" so it can be found and loaded successfully.
```

## Phase Options

Each phase has its own unique set of options, which are further organized inside additional sub-dictionaries. Here is an example of a phase-specific dictionary with the 3 possible sub-categories already defined:

```python
phase_info['climb1'] = {
    'subsystem_options': {},
    'user_options': {},
    'initial_conditions': {},
}
```

The 'subsystem_options' dictionary contains options for all subsystems.
The 'user_options' dictionary includes all options that define the phase and its boundary conditions.
The 'initial_conditions' dictionary allows you to set initial values for any variables that are under control of the optimizer, including states, optimized controls, and time.

The following sections outline important options available for each of these categories. A full description of every user option in the phase_info for each of Aviary's phase builders is given in the [Complete `phase_info` Reference.](../phase_info_detailed.ipynb).

If you have a phase_info that was built in an version of Aviary prior to version 1.0, check the [legacy `phase_info` conversion guide](../../misc_resources/phase_info_conversion.ipynb) to learn what has changed.

## Subsystem Options

Here subsystem-specific options can be defined, which only apply to the specific phase they are present in. What options are available depends on which specific subsystems you are including with your model. Available options for the core Aviary subsystems are defined in their respective documentation pages in the [User Guide](../../user_guide/subsystems.md).

## User Options

The user options format supports (and enforces) the inclusion of an OpenMDAO unit string when specifying any quantity that has a unit. These take the format of a tuple, such as "(500, 's')" to represent "500 seconds". If an option requires multiple value, a nested tuple is used, with the list of values stored in their own tuple: "((300.0, 900.0), 's')". For options whose values do not have a unit, you can either skip the units definition entirely, just directly providing the desired value, or you can specify units of "unitless" for clarity.

Aviary includes several sets of equations of motion to define flight physics, each with their own additional set of available options. Thus, the full set of valid options that can be specified in `phase_info` can differ based on your problem setup. The following section describes the general patterns that globally apply to all Aviary models.<!-- TODO: add documentation for EOM-specific phase_info options. Each EOM should probably have its own page in the User Guide, which is a good place for this info. Link to those files here once done. -->

### General phase settings

These options can be specified for any mission phase.
<!-- TODO: include default values for each setting, as well as if they are optional -->
<!-- TODO: reduce verbosity of this section and standardize - make it more like a docstring. Name, default value, description of variable, list possible options (or data type if no set list of allowable options) and what each does -->

- **{glue:md}`num_segments`**: The number of segments in the dymos transcription.

- **{glue:md}`order`**: The order of polynomials for the dymos transcription.

- **{glue:md}`throttle_enforcement`**: Enforce throttle as a "path_constraint" (default), "boundary_constraint", a "control" (which can be optimized or set to a fixed value), or "bounded" via a solver upper and lower bounds (not recommended).

- **{glue:md}`throttle_allocation`**: Pick method used to solve required throttles when using multiple unique engine types. Possible options are "fixed" for user-specified allocations, "static" where the optimizer picks a single allocation for the whole phase, or "dynamic" where the optimizer chooses an allocation at each trajectory point.

- **{glue:md}`no_climb`**: Set to True to prevent the aircraft from climbing during the phase. This option can be used to prevent unexpected climb during a descent phase. This is only available when using the height_energy equations of motion.

- **{glue:md}`no_descent`**: Set to True to prevent the aircraft from descending during the phase. This can be used to prevent unexpected descent during a climb phase. This is only available when using the height_energy equations of motion.

### Specialized phase settings

These options are unique to specific types of phases, or have other limitations on their use.

- **{glue:md}`reserve`**: When True, this phase is counted as part of the reserve mission. These phases should appear sequentially after the final flight phase of the main mission.

- **{glue:md}`ground_roll`**: When True, restrict the problem to the ground plane. This will remove certain states/controls, and is only used in takeoff and landing phases when using the 2DOF equations of motion.

- **{glue:md}`required_available_climb_rate`**: Adds a lower bound path constraint on altitude_rate_max, which is the maximum rate that the aircraft could climb at any point in the phase.

- **{glue:md}`target_distance`**: Total mission distance constraint. This should be set in the final flight phase only.

- **{glue:md}`constraints`**: Dictionary of custom constraints applied to dynamic variables in the phase.

### State variables

These options provide information about state variables, or variables that are summed over the course of a phase by Dymos. Which state variables exist in a phase is dependent on what kind of phase it is and which equations of motion are being used. Here, the options for the "mass" state are described. When defining options for a different state variable, the same set of options are available, but the variable name at the beginning of the option must be swapped out for the state variable of interest (such as `mass_initial` becoming `altitude_initial`).

- **{glue:md}`mass_initial`**: Mass at the start of the phase. When unspecified, the optimizer controls the value. When specified, a constraint is created on the initial mass.

- **{glue:md}`mass_final`**: Mass at the end of the phase. When unspecified, the optimizer controls the value. When specified, a constraint is created on the final mass.

- **{glue:md}`mass_bounds`**: Tuple of upper and lower bounds for all values of mass in the phase. The default of None for either means that bound will not be declared.

- **{glue:md}`mass_ref`**: Multiplicative scale factor "ref" for mass. Default is 1.0

- **{glue:md}`mass_ref0`**: Additive scale factor "ref0" for mass. Default is None.

- **{glue:md}`mass_defect_ref`**: Multiplicative scale factor "ref" for defect constraint. Default is None, which means the ref and defect_ref are the same.

- **{glue:md}`mass_constraint_ref`**: Multiplicative scale factor "ref" for boundary constraint. If unspecified, then the value in `mass_final` will be used.

- **{glue:md}`mass_solve_segments`**: When True, a solver will be used to converge the mass collocation defects within a segment. Note that the state continuity defects between segments will still be handled by the optimizer.

### Controls

These options provide information about control variables, or variables that are directly controlled by the optimizer. Like state variables, control variables for a given phase is also dependent on what kind of phase it is and which equations of motion are being used. The naming scheme is also similar, with the control variable of interest forming the first part of the option's name.

- **{glue:md}`altitude_optimize`**: When True, the optimizer will set this value. When False, the initial value for all nodes can be set in the initial_conditions section of the phase.

- **{glue:md}`altitude_initial`**: Altitude at the start of the phase. If altitude_optimize is True, specifying this will create a constraint. Otherwise, it serves as an initial value for the first point. When unspecified, the linkage constraint with the upstream phase will drive this value if altitude_optimize is True.

- **{glue:md}`altitude_final`**: Altitude at the end of the phase. If altitude_optimize is True, specifying this will create a constraint. Otherwise, it serves as an initial value for the first point. When unspecified, the linkage constraint with the downstream phase will drive this value if altitude_optimize is True.

- **{glue:md}`altitude_bounds`**: Tuple of upper and lower bounds for all values of altitude in the phase. The default of None for either means that bound will not be declared.

- **{glue:md}`altitude_ref`**: Multiplicative scale factor "ref" for altitude. Default is 1.0

- **{glue:md}`altitude_ref0`**: Additive scale factor "ref0" for altitude. Default is None.

- **{glue:md}`altitude_polynomial_order`**: The order of polynomials for interpolation in the transcription. Default is None, which does not use a polynomial.

### Time
- **{glue:md}`time_initial`**: Value of "time" at the start of the phase. When unspecified, the value is determined by the optimizer.

- **{glue:md}`time_duration`**: Duration of the phase. When unspecified, the value is determined by the optimizer.

- **{glue:md}`time_initial_bounds`**: Upper and lower bounds for time_initial. Ignored if "time_initial" is specified. 

- **{glue:md}`time_duration_bounds`**: Upper and lower bounds for time_duration. 

- **{glue:md}`time_initial_ref`**: Multiplicative scale factor "ref" for time_initial. Default is 1.0

- **{glue:md}`time_duration_ref`**: Additive scale factor "ref" for time_duration. Default is None.

### Specialized 2DOF phase settings
<!-- TODO: see earlier TODO to consider moving these kinds of EOM-specific options to their own doc page -->

When using the 2DOF equations of motion, there are several phase options that are only available in specific phases. Future refactor may change some of these to be more consistent with the phase_options listed above in naming and function.

- **{glue:md}`alt`**: Constant altitude for this phase. Only available in accel phases (i.e, phases with "accel" in their name).

- **{glue:md}`EAS_limit`**: Maximum speed constraint in this phase. Only available in descent phases (i.e, phases with "desc" in their name).

- **{glue:md}`EAS_target`**: Target airspeed for the balance in this phase. Only available in climb phases (i.e, phases with "climb" in their name).

- **{glue:md}`mach_cruise`**: Used to define the mach constraint at the end of climb, the fixed mach during cruise, or the overall mach constraint in descent. Only available in climb, cruise, or descent phases (i.e, phases with "climb", "cruise", or "desc" in their name).

- **{glue:md}`normal_ref`**: Multiplicative scale factor "ref" for the normal force constraint. Only available in rotation phases (i.e, phases with "rotation" in their name).

- **{glue:md}`normal_ref0`**: Additive scale factor "ref0" for the normal force constraint. Only available in rotation phases (i.e, phases with "rotation" in their name).

- **{glue:md}`pitch_constraint_bounds`**: Lower and upper bounds of the pitch constraint. Only available in ascent phases (i.e, phases with "ascent" in their name).

- **{glue:md}`pitch_constraint_ref`**: Multiplicative scale factor "ref" for the pitch constraint. Only available in ascent phases (i.e, phases with "ascent" in their name).

- **{glue:md}`target_mach`**: When True, add a constraint with the value specified in "mach_cruise". Only available in climb and descent phases (i.e, phases with "climb" or "desc" in their name).